# Simulated Live Financial Data - Apple

In [4]:
import yfinance as yf
import pandas as pd

# Fetch historical data
symbol = 'AAPL'
df = yf.download(symbol, start="2020-01-01", end="2025-01-01", progress=False)

# Reset the index to get 'Date' as a column
df.reset_index(inplace=True)

# Add the Ticker column manually
df['Symbol'] = symbol

# Reorder columns
df = df[['Symbol', 'Date', 'Close', 'High', 'Low', 'Open', 'Volume']]
df.columns = ['Symbol', 'Date', 'Close', 'High', 'Low', 'Open', 'Volume']

# Display result
print(df.head())


  Symbol       Date      Close       High        Low       Open     Volume
0   AAPL 2020-01-02  72.716080  72.776606  71.466820  71.721026  135480400
1   AAPL 2020-01-03  72.009125  72.771752  71.783969  71.941336  146322800
2   AAPL 2020-01-06  72.582916  72.621654  70.876083  71.127873  118387200
3   AAPL 2020-01-07  72.241547  72.849224  72.021231  72.592594  108872000
4   AAPL 2020-01-08  73.403633  73.706264  71.943744  71.943744  132079200


In [5]:

from kafka import KafkaProducer
import time
import json

# Define Kafka server and topic
KAFKA_SERVER = '8e35f34f36db:9093'
KAFKA_TOPIC = 'kafka-apple'

# Initialize Kafka producer
producer = KafkaProducer(
    bootstrap_servers=KAFKA_SERVER,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')  # serialize to JSON
)

# Send each row of the DataFrame to Kafka
try:
    print(f"Producing messages to Kafka topic: {KAFKA_TOPIC}")
    for _, row in df.iterrows():
        message = row.to_dict()
        print(message)
        # Optionally add symbol
        # message['symbol'] = symbol
        # Convert Timestamp to string
        message['Date'] = message['Date'].strftime('%Y-%m-%d')

        producer.send(KAFKA_TOPIC, message)
        print(f"Sent: {message}")
        time.sleep(1)  # Simulate delay like real-time stream

except KeyboardInterrupt:
    print("Stopped producing messages.")
finally:
    producer.close()

Producing messages to Kafka topic: kafka-apple
{'Symbol': 'AAPL', 'Date': Timestamp('2020-01-02 00:00:00'), 'Close': 72.71607971191406, 'High': 72.77660582997152, 'Low': 71.46681974859514, 'Open': 71.7210264890596, 'Volume': 135480400}
Sent: {'Symbol': 'AAPL', 'Date': '2020-01-02', 'Close': 72.71607971191406, 'High': 72.77660582997152, 'Low': 71.46681974859514, 'Open': 71.7210264890596, 'Volume': 135480400}
{'Symbol': 'AAPL', 'Date': Timestamp('2020-01-03 00:00:00'), 'Close': 72.00912475585938, 'High': 72.7717522953066, 'Low': 71.78396939069293, 'Open': 71.94133580542943, 'Volume': 146322800}
Sent: {'Symbol': 'AAPL', 'Date': '2020-01-03', 'Close': 72.00912475585938, 'High': 72.7717522953066, 'Low': 71.78396939069293, 'Open': 71.94133580542943, 'Volume': 146322800}
{'Symbol': 'AAPL', 'Date': Timestamp('2020-01-06 00:00:00'), 'Close': 72.58291625976562, 'High': 72.62165386110323, 'Low': 70.87608272259153, 'Open': 71.12787343706493, 'Volume': 118387200}
Sent: {'Symbol': 'AAPL', 'Date': '2